In [1]:
import os

In [2]:
%pwd

'c:\\End_To_End_deeplearning\\DeepLearning_EndtoEnd_Project\\research'

In [3]:
#to go back one folder
os.chdir("../")

In [4]:
%pwd

'c:\\End_To_End_deeplearning\\DeepLearning_EndtoEnd_Project'

In [26]:
from dataclasses import dataclass
from pathlib import Path

#entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [27]:
from src.CNNClassifier.constants import *
from src.CNNClassifier.utils.common import read_yaml,create_directories

In [28]:
#update configuration manager
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH

    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    #data ingestion function
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        #return type of function
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip=config.unzip_dir
        )

        return data_ingestion_config

        

In [29]:
#update components
import urllib.request as request # to download the data
import zipfile #unzip the file
from src.CNNClassifier import logger #custom logger
from src.CNNClassifier.utils.common import get_size #to se the data soze

In [30]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config
    
    #to dowload the file from github
    def download_file(self):
        #if data set is not in local system
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            #to save the actions in log file
            logger.info(f"{filename} download! with following info: \n{headers}")
        #if already stored in local
        else:
            #to save the actions in log file
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the directory
        fUnction returns None
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [31]:
#pipeline
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-15 13:22:48,250: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-15 13:22:48,252: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-15 13:22:48,252: INFO: common: created directory at: artifacts]
[2023-07-15 13:22:48,252: INFO: common: created directory at: artifacts/data_ingestion]


TypeError: __init__() got an unexpected keyword argument 'unzip'